# ДЗ 8

В 2023 году проходил контест от Телеграма. Нужно было определять язык программирования по сниппету(*)

https://contest.com/docs/ML-Competition-2023

В этой домашке нужно будет решить упрощенную версию этого контеста.

Ограничения:

- Не более 4096 символов на входе
- Не гарантируется, что код на входе синтаксически корректный
- Не гарантируется, что на входе код

Можно использовать любые модели и любые данные.

Важно текстом описать ход решения - как собирали датасет, как процессили данные, какие метрики, какие модели выбрали и почему.

Существует небольшой отложенный датасет, на котором можно будет проверить свое решение после дедлайна.

**Совет: если будете скачивать чатики через скрипты, лучше заведите отдельный аккаунт для этого.**



Запарсил гитхаб, обучил дефолтный катбуст

## Easy

Учимся отличать код на Python, C++ и JavaScript от обычного текста на английском языке (Other).

Собираем датасет, учим модели, собираем все в скрипт

Должно работать так:

```
$ python main.py python_code.txt
Python
```

Оставить по 3 примера на каждый язык, на которых проверяете решение. Текстовые файлики приложить в репозиторий.


In [1]:
import pandas as pd
import os
from script import extol, languages

In [2]:
base_dir = '../data/files'
file_list = [f for f in os.listdir(base_dir) if os.path.isfile(os.path.join(base_dir, f))]
file_info = {}
for file in file_list:
    file_ext = os.path.splitext(file)[1][1:]
    with open(os.path.join(base_dir, file), 'rb') as f:
        content = f.read().decode('utf-8')

    file_info[file] = [content, extol[file_ext]]

In [3]:
df = pd.DataFrame(file_info).T
df.columns = ['code', 'lang']
df.sample(3)

,code,lang
DL3032.hs,module Hadolint.Rule.DL3032 (rule) where\n\nim...,haskell
amlogic.yaml,# SPDX-License-Identifier: GPL-2.0\n%YAML 1.2\...,yaml
pctrl.yaml,# SPDX-License-Identifier: (GPL-2.0-only OR BS...,yaml


In [4]:
print(df['code'].iloc[0])

const markdownIt = require("markdown-it")
const markdownItAnchor = require("markdown-it-anchor")
const markdownItReplaceLink = require('markdown-it-replace-link')
const slugify = require("slugify")
const uri = require('fast-uri')
const i18n = require('eleventy-plugin-i18n')
const fs = require("fs")
const path = require("path")
const pluginRss = require('@11ty/eleventy-plugin-rss')
const { JSDOM } = require('jsdom')


// The implementation of Glossary feature
const md = new markdownIt()
const glossaryMarkdownContent = fs.readFileSync(path.resolve(__dirname, '../docs/GLOSSARY.md'), 'utf8')
const glossaryHtmlContent = md.render(glossaryMarkdownContent)
const glossaryDOM = new JSDOM(glossaryHtmlContent)
const glossaryDocument = glossaryDOM.window.document
const glossary = require('./src/_data/glossary.json')

glossary.forEach(item => {
  const headers = Array.from(glossaryDocument.querySelectorAll("h2"))
  const matchingHeader = headers.find(header => header.textContent.trim() === item.def

In [5]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.code, df.lang)
X_train = pd.DataFrame(X_train, columns=["code"])
X_test = pd.DataFrame(X_test, columns=["code"])

In [7]:
clf = CatBoostClassifier(text_features=["code"])

In [8]:
# clf.fit(X_train, y_train)
clf.load_model("classifier")

In [9]:
dfs = []
dir = '../chats'
for f in os.listdir(dir):
    path = os.path.join(dir, f)
    df = pd.read_json(path)
    df['lang'] = '_'.join(map(str.lower, f.split('_')[:-1]))
    dfs.append(df)

In [10]:
df_1 = pd.concat(dfs)
df_1.sample(3)
df_1.columns = ['code', 'lang']
df_1['has_code'] = df_1.code.str.contains('```')

In [11]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=languages.keys())

print(f"Accuracy: {accuracy}\nClassification Report: \n{report}")

Accuracy: 0.972972972972973
Classification Report: 
              precision    recall  f1-score   support

         C++       0.96      1.00      0.98        23
        yaml       1.00      1.00      1.00        22
        bash       0.97      1.00      0.99        38
    markdown       0.97      1.00      0.98        32
           C       0.95      0.95      0.95        22
      kotlin       0.94      0.77      0.85        22
     haskell       0.94      0.94      0.94        31
        text       1.00      1.00      1.00        23
        java       0.97      1.00      0.98        30
  javascript       1.00      1.00      1.00        32
      python       1.00      1.00      1.00        21

    accuracy                           0.97       296
   macro avg       0.97      0.97      0.97       296
weighted avg       0.97      0.97      0.97       296



In [12]:
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[23  0  0  0  0  0  0  0  0  0  0]
 [ 0 22  0  0  0  0  0  0  0  0  0]
 [ 0  0 38  0  0  0  0  0  0  0  0]
 [ 0  0  0 32  0  0  0  0  0  0  0]
 [ 0  0  0  0 21  0  0  0  1  0  0]
 [ 1  0  1  0  1 17  2  0  0  0  0]
 [ 0  0  0  1  0  1 29  0  0  0  0]
 [ 0  0  0  0  0  0  0 23  0  0  0]
 [ 0  0  0  0  0  0  0  0 30  0  0]
 [ 0  0  0  0  0  0  0  0  0 32  0]
 [ 0  0  0  0  0  0  0  0  0  0 21]]


In [13]:
df_1.lang.value_counts()

lang
cplusplus        5250
yaml             3000
csharp           3000
typescript       3000
sql              3000
python           3000
powershell       3000
php              3000
java             3000
haskell          3000
javascript       2711
1s_enterprise    2000
ruby             2000
dart             2000
delphi           2000
perl             2000
assembly         2000
rust             2000
ocaml            1000
pascal           1000
cobol            1000
clojure          1000
batch            1000
r                1000
mysql            1000
scala            1000
solidity         1000
basic            1000
swift            1000
thedevs          1000
autohotkey       1000
nim              1000
css              1000
matlab           1000
fortran          1000
d                1000
elixir           1000
elm              1000
erlang           1000
eth              1000
forth            1000
fsharp           1000
c                1000
go               1000
crystal          1000
julia

In [14]:
df_1.loc[df_1.lang == 'cplusplus', 'lang'] = 'C++'
df_1.loc[df_1.lang == 'c', 'lang'] = 'C'

In [15]:
df_test = df_1[df_1.lang.isin(languages.keys()) & df_1.has_code]
df_test.lang.value_counts()

lang
C++           154
yaml           68
haskell        56
python         52
java           31
javascript     18
C              15
kotlin         13
Name: count, dtype: int64

In [16]:
df_test.sample()

,code,lang,has_code
864,insert at least works like this\n```->Execute(...,C++,True


In [17]:
X_test = pd.DataFrame(df_test.code)
X_test.columns = ['code']
X_test.sample()

,code
997,Привет всем.\nЕсть вот такой вот код — https:/...


In [18]:
y_pred = clf.predict(X_test)
y_test = df_test.lang

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=languages.keys())

print(f"Accuracy: {accuracy}\nClassification Report: \n{report}")

Accuracy: 0.14250614250614252
Classification Report: 
              precision    recall  f1-score   support

         C++       0.17      0.07      0.10        15
        yaml       0.33      0.03      0.06       154
        bash       0.00      0.00      0.00         0
    markdown       0.84      0.29      0.43        56
           C       0.57      0.26      0.36        31
      kotlin       0.06      0.28      0.11        18
     haskell       0.09      0.85      0.15        13
        text       0.00      0.00      0.00         0
        java       0.27      0.23      0.25        52
  javascript       0.00      0.00      0.00         0
      python       0.00      0.00      0.00        68

    accuracy                           0.14       407
   macro avg       0.21      0.18      0.13       407
weighted avg       0.33      0.14      0.15       407



c:\Users\bentodan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\bentodan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\bentodan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, m

Действительно, работает сильно хуже, но в задании не шло изначально речи, что нужно именно на этот датасет валидироваться (было написано, что нужно уметь определять язык)

In [ ]:
with open('script.py', 'rb') as f:
    py = f.read().decode('utf-8')
print(py[:1000])

import requests
import os

token = "it dissapeared:("
languages = {'C++': 'cpp', 'yaml': 'yaml', 'bash': 'sh', 'markdown': 'md', 'C': 'c', 
             'kotlin': 'kt', 'haskell': 'hs', 'text': 'txt', 'java': 'java', 'javascript': 'js', 'python': 'py'}
extensions = tuple(['.' + el for el in languages.values()])
counts = {'C++': 0, 'yaml': 0, 'bash': 0, 'markdown': 0, 'C': 0, 'kotlin': 0,
      'haskell': 0, 'text': 0, 'java': 0, 'javascript': 0, 'python': 0}
extol = {'cpp': 'C++', 'yaml': 'yaml', 'sh': 'bash', 'md': 'markdown', 'c': 'C', 
             'kt': 'kotlin', 'hs': 'haskell', 'txt': 'text', 'java': 'java',  'js': 'javascript', 'py': 'python'}
base_dir = "..\\data\\files"


def search_github_repos(language, per_page=10):
    url = f"https://api.github.com/search/repositories?q=language:{language}&per_page={per_page}&page=6"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()['items']
    return []

headers = {'


In [ ]:
test = pd.DataFrame([py], columns=["code"])
clf.predict(test)

array([['python'],
       ['C++']], dtype=object)

In [ ]:
clf.save_model("classifier")

In [ ]:
!python main.py python_code.txt

python


## Medium

Количество языков расширяется до 10

- Python
- C++
- JavaScript
- Java
- Yaml
- Bash
- Markdown
- C
- Kotlin
- Haskell

Какие выводы можно сделать по данным? 

Вывод: катбуст крутой, но если задача одна, а понята иначе, результат будет не очень